# Results

In [ ]:
import re
import pandas as pd

def read_logs(log_paths):
    c = []
    for exp in log_paths:
        with open(exp) as infile:
            loglines = infile.read().splitlines()

        mem_size = int(re.findall(r"memory_size=(\d+)", loglines[0])[0])
        final_acc = float(re.findall(r"Acc: \[(.*?)\]", loglines[-1])[0])
        compressor = re.findall(r"compressor=\'(.*?)\'", loglines[0])[0]
        encoder = re.findall(r"encoder=\'(.*?)\'", loglines[0])[0]
        encoding_block = float(re.findall(r"backbone_block=([\d.]+)", loglines[0])[0])
        backbone = re.findall(r"backbone=\'(.*?)\'", loglines[0])[0]
        dataset = re.findall(r"dataset=\'(.*?)\'", loglines[0])[0]
        filename = exp.parents[0].name

        compressor_param = None

        if compressor == 'autoencoder':
            compressor = 'convae'

        if compressor == 'thinning':
            compressor_param = float(re.findall(r"compression_factor=(.*?),", loglines[0])[0])
        elif compressor == 'quantization':
            compressor_param = int(re.findall(r"n_states=(\d+)", loglines[0])[0])
            try:
                strategy = re.findall(r"strategy=\'(.*?)\'", loglines[0])[0]
                if strategy in ['tiny_imagenet_transfer', 'cifar10_transfer', 'cifar100_transfer']:
                    compressor_new = 'quantization transfer'
                elif strategy == 'local':
                    compressor_new = 'quantization local'
                else:
                    raise ValueError('Unknown Quantization Stragegy: ' + strategy)
            except IndexError:
                compressor_new = 'quantization local'
            compressor = compressor_new
        elif compressor == 'convae':
            compressor_param = int(re.findall(r"latent_channels=(\d+)", loglines[0])[0])
        elif compressor == 'fcae':
            compressor_param = int(re.findall(r"bottleneck_neurons=(\d+)", loglines[0])[0])
        elif compressor == 'none':
            compressor_param = 0
        else:
            raise ValueError(f'Unknown Compressor: {compressor}')

        encoder = 'cutr' if encoder == 'cutr34' else encoder

        c.append({
            'backbone': backbone,
            'mem_size': mem_size,
            'final_acc': final_acc,
            'encoder': encoder,
            'encoding_block': encoding_block,
            'compressor': compressor,
            'compressor_param': compressor_param,
            'dataset': dataset,
            'filename': filename,
        })
        
    df = pd.DataFrame.from_records(c)
    df.drop(df.loc[
        (df['compressor']=='fcae') | (df['compressor']=='quantization local')
        ].index, inplace=True)
    
    return df

In [ ]:
from pathlib import Path

# log_paths = [Path(d, 'checkpoint.log') for d in Path('../logs/vs_acae_remind_v4').glob('[!.]*')]
paths_a = [Path(d, 'checkpoint.log') for d in Path('/Users/marwei/AFS/home/urz/m/marwei/Logs/IDEAL/cifar10_vs_remind_setting_A/').glob('[!.]*')]
paths_b = [Path(d, 'checkpoint.log') for d in Path('/Users/marwei/AFS/home/urz/m/marwei/Logs/IDEAL/cifar10_vs_remind_setting_B/').glob('[!.]*')]
paths_c = [Path(d, 'checkpoint.log') for d in Path('/Users/marwei/AFS/home/urz/m/marwei/Logs/IDEAL/cifar10_vs_remind_setting_C/').glob('[!.]*')]
df_a = read_logs(paths_a)
df_b = read_logs(paths_b)
df_c = read_logs(paths_c)


In [ ]:
c = []
c.append({
        'mem_size': 24000,
        'final_acc': 0.484,
        'encoding_block': 2.5,
        'compressor': 'ACAE Remind',
        'compressor_param': 0,
        'dataset': 'CIFAR10',
    }
)
c.append({
        'mem_size': 24000,
        'final_acc': 0.452,
        'encoding_block': 3.5,
        'compressor': 'REMIND',
        'compressor_param': 0,
        'dataset': 'CIFAR10',
    }
)
c.append({
        'mem_size': 8000,
        'final_acc': 0.643,
        'encoding_block': 0,
        'compressor': 'Resize 8x8',
        'compressor_param': 0,
        'dataset': 'CIFAR10',
    }
)
df_lit = pd.DataFrame.from_records(c)

df_a = pd.concat((df_a, df_lit))
df_b = pd.concat((df_b, df_lit))
df_c = pd.concat((df_c, df_lit))

In [ ]:
from plot_utils import science_template

compressor_names = {
    'none': 'No Compressor',
    'thinning': 'Thinning',
    'quantization transfer': 'Quantization',
    'convae': 'Autoencoder',
    'ACAE Remind': 'ACAE Remind',
    'REMIND': 'REMIND',
    'Resize 8x8': 'Smaller Is Better'
}

encoder_names = {
    'cutr': 'FETCH',
    'none': 'no Fixed Encoder'
}

enc_dict = {
    0: ('solid', 'circle', 'No Encoder'),
    2.5: ('dash', 'square', 'Encoded at Block 2'),
    3.5: ('dot', 'x', 'Encoded at Block 3'),
}

colors = science_template['layout']['colorway'][:6] + science_template['layout']['colorway'][7:]
compressor_order = ['none', 'quantization transfer', 'thinning', 'convae', 'ACAE Remind', 'REMIND', 'Resize 8x8']
com_dict = {k: v for k, v in zip(compressor_order, colors)}
del colors, compressor_order

df_a['compressor_name'] = df_a['compressor'].map(compressor_names)
df_a['encoder_name'] = df_a['encoder'].map(encoder_names)

df_b['compressor_name'] = df_b['compressor'].map(compressor_names)
df_b['encoder_name'] = df_b['encoder'].map(encoder_names)

df_c['compressor_name'] = df_c['compressor'].map(compressor_names)
df_c['encoder_name'] = df_c['encoder'].map(encoder_names)

In [ ]:
import plotly.graph_objects as go

def create_figure(fig, df, row, col):
  
    # add the actual traces
    for this_encoding_block in df['encoding_block'].unique():
        for this_compressor in df['compressor'].unique():
            view = df.loc[
                (df['encoding_block'] == this_encoding_block) & (df['compressor'] == this_compressor)
            ].sort_values('mem_size')
            if len(view) == 0:
                continue
            if len(view) == 1:
                mode = 'markers'
            else:
                mode = 'lines+markers'
            
            if this_compressor in ['ACAE Remind', 'REMIND', 'Resize 8x8']:
                marker_line_dict = {'width': 1, 'color': 'black'}
            else:
                marker_line_dict = {}
            fig.add_trace(
                go.Scatter(
                    x=view['mem_size'],
                    y=view['final_acc'],
                    mode=mode,
                    name=view.compressor_name.unique()[0],
                    showlegend=False,
                    line={
                        'dash': enc_dict[this_encoding_block][0],
                        'color': com_dict[this_compressor],
                        'width': 2
                    },
                    marker={
                        'symbol': enc_dict[this_encoding_block][1],
                        'color': com_dict[this_compressor],
                        'size': 5,
                        'line': marker_line_dict
                    },
                    text=[f"encoding_block={this_row['encoding_block']}<br>k={this_row['compressor_param']}" for _, this_row in view.iterrows()],
                    hovertemplate='N=%{x}<br>'+\
                                  'Acc.=%{y}<br>'+\
                                  '%{text}',
                ),
                row=row, col=col,
            )

In [ ]:
def add_legend(fig, df_lit, col):
    x_coord = 100,
    y_coord = 100
    for key, val in enc_dict.items():
        fig.add_trace(
            go.Scatter(
                x=[x_coord],
                y=[y_coord],
                mode='lines',
                line={
                    'dash': val[0],
                    'color': 'black'
                },
                showlegend=True,
                name=val[2],
                legendgroup='blocks',
                legendgrouptitle_text="Encoding-Strategy"
            ),
            row=1, col=col
        )
        fig.add_trace(
            go.Scatter(
                x=[x_coord],
                y=[y_coord],
                mode='markers',
                line={
                    'dash': val[0],
                    'color': 'black'
                },
                marker={
                    'symbol': val[1],
                    'color': 'black'
                },
                showlegend=True,
                name=val[2],
                legendgroup='blocks',
                legendgrouptitle_text="Encoding-Strategy"
            ),
            row=1, col=col
        )

    for key in  ['none', 'quantization transfer', 'thinning', 'convae']:
        name = compressor_names[key]
        fig.add_trace(
            go.Scatter(
                x=[x_coord],
                y=[y_coord],
                mode='lines',
                line={
                    'color': com_dict[key],
                    'width': 20
                },
                showlegend=True,
                name=name,
                legendgroup='compressor',
                legendgrouptitle_text='Compression-Strategy'
            ),
            row=1, col=col
        )
        
    for key in  ['ACAE Remind', 'REMIND', 'Resize 8x8']:
        name = compressor_names[key]
        encoding_block = df_lit.loc[df_lit['compressor']==key, 'encoding_block'].item()
        fig.add_trace(
            go.Scatter(
                x=[x_coord],
                y=[y_coord],
                mode='markers',
                marker={
                    'color': com_dict[key],
                    'symbol': enc_dict[encoding_block][1],
                    'size': 7,
                    'line': {
                        'width': 1,
                        'color': 'black'
                    },
                },
                showlegend=True,
                name=name,
                legendgroup='compressor',
                legendgrouptitle_text='Compression-Strategy'
            ),
            row=1, col=col
        )

In [ ]:

from plotly.subplots import make_subplots

fig=make_subplots(
    rows=1,
    cols=3,
    subplot_titles=['Setting A', 'Setting B', 'Setting C'],
    shared_yaxes=True,
    horizontal_spacing=0.02
)

create_figure(fig, df_a, row=1, col=1)
create_figure(fig, df_b, row=1, col=2)
create_figure(fig, df_c, row=1, col=3)
add_legend(fig, df_lit, col=3)

# make the legend manually


fig.update_layout(
    template=science_template,
    yaxis_range=(0.15, 0.75),
    width=810,
    height=400
)

fig.update_yaxes(
        title='Accuracy',
        row=1,
        col=1,
    )
for i in range(1, 4):
    fig.update_xaxes(
        range=(2.3, 4.7),
        type='log',
        title='N',
        row=1,
        col=i,
        title_standoff=3,
    )
    

fig.show()
fig.write_image('../plots/vs_remind.pdf')